In [36]:
import numpy as np
import pandas as pd
import xlrd
from collections import defaultdict
from IPython.display import display

class TimeTable_GA():
    
    def _init_(self):
        self.testCase = "" #name of the testCase
        self.TotalClassInWeek = 0.5 #total classes that are occuring in a week
        self.TeacherCount = 0
        self.CoursesOffered = np.empty()
        self.ClassScheduled = np.empty()
        self.TotalRooms = np.empty()
        self.GroupsCount = 0
        self.days = np.empty()
        self.time = np.empty()
        self.Population = np.empty() #stored the generated Population
    
    #function to display
    def Display(self, array):
        df = pd.DataFrame(array)
        display(df)
        
    #function that reads a an excel file containing the test case
    def readTestFile(self, filename):
        self.testCase = filename
        
        #Reads the courses headins from the fileName and defines the list of courses offered
        print ("Storing the Courses Offered:")
        testCase = xlrd.open_workbook(self.testCase)
        sheet = testCase.sheet_by_index(1)
        self.CoursesOffered = sheet.row_values(6)
        self.CoursesOffered = [var for var in self.CoursesOffered if var] 
        self.Display(self.CoursesOffered)
        self.CoursesOffered = np.insert(self.CoursesOffered, 0, '', axis=0)
        
         #define the classes that are scheduled
        print ("Storing the Classes that are to be scheduled: ")       
        mainList = []
        index=7
        while index<=52:
            tech_list= sheet.row_slice(rowx=index,start_colx=0,end_colx=7)
            dur_list= sheet.row_slice(rowx=index+1,start_colx=0,end_colx=7)
            temp_lst1=[]
            temp_lst2=[]
            for index2 in range(7):
                if index2==0:
                    temp_lst1.append(tech_list[index2].value)
                    temp_lst2.append(0)
                else:
                    temp_lst1.append(tech_list[index2].value)
                    temp_lst2.append(dur_list[index2].value)
            temp_lst1=np.array(temp_lst1)
            temp_lst2=np.array(temp_lst2)
            for index3 in range(1,7):
                mainList.append([temp_lst1[0], self.CoursesOffered[index3],temp_lst1[index3],temp_lst2[index3]])
            index+=2
        
        temp_Array4 = []
        length=len(mainList)
        b=0
        c=0
        for i in range(length):
            temp=mainList[i]
            if float(temp[3])==1.5:
                b+=1
                temp_Array4.append(mainList[i])
                temp_Array4.append(mainList[i])
            elif float(temp[3])==1:
                c+=1
                temp_Array4.append(mainList[i])
                temp_Array4.append(mainList[i])
                temp_Array4.append(mainList[i])
        self.ClassScheduled = temp_Array4
        self.Display (self.ClassScheduled)
        self.ClassScheduled = np.array(self.ClassScheduled)
        
        #Defining Rooms
        print ("Defining rooms")
        cell = sheet.cell(3,1)
        self.TotalRooms = np.arange(1, int(cell.value)+1)
        self.Display(self.TotalRooms)
        
        self.defineDays()
        self.defineTimeIntervals()
    
    #Assigning Days in the week for Time Table
    def defineDays(self):   
        print ("Days in the week")
        self.days = np.arange(1,6) #(Mon - Friday)
        self.days = np.array(self.days)
        print (self.days)
        
    #Assiging time intervals for each day
    def defineTimeIntervals(self):
        print ("Time Intervals")
        self.time = np.arange(9,16.1,0.5) #assigning time
        self.time = np.array(self.time)
        print (self.time)
        

    def pickRandom (self, array):
        return np.random.choice(array)
        
    #Function that generate a chromosome
    def generateChromosome (self):    
        tempArray = []
        for i in range (len(self.ClassScheduled)):
            temp = list(self.ClassScheduled[i])
            #print (i," ", temp)
            
            if (float(temp[3])==1.5): #If the duration of the class is 1.5 hours
                randomDay = self.pickRandom(self.days)
                randomTime = self.pickRandom(self.time[:-1])
                randomRoom = self.pickRandom(self.TotalRooms)
                
                if (randomDay == 5): #if the randomly chosen day is Friday
                    times_ = [12,12.5,13,13.5,14]
                    while randomTime in times_:
                        randomTime = self.pickRandom(self.time[:-1])
                        
                endDuration = randomTime + 1.5      
                temp.extend([randomRoom, randomDay, randomTime, endDuration])
                
            else: #if the duration of the claass is standard 1 hour
                randomDay = self.pickRandom(self.days)
                randomTime = self.pickRandom(self.time)
                randomRoom = self.pickRandom(self.TotalRooms)
                
                if (randomDay == 5): #if the day chosen is Friday
                    times_ = [12.5,13,13.5,14]
                    while randomTime in times_:
                        randomTime = self.pickRandom(self.time)
                        
                endDuration = randomTime + 1.5        
                temp.extend([randomRoom, randomDay, randomTime, endDuration])
            print (i, " ", temp)
            tempArray.append(temp)
        return np.array(tempArray)
    
    def overlap(self, array1):
        v=0
        array=np.array(array1,dtype=np.float32)
        nexamples=array.shape[0]
        #print "e ",nexamples
        for i in range(nexamples):
            j=i
            while (j<nexamples):
                if i!=j:
                    if (array[i,0]>array[j,0] and array[i,0]<array[j,1]) or (array[i,1]>array[j,0] and array[i,1]<array[j,1]):
                        v+=1
                j+=1
        return v
    

    def same_course(self, array):
        mydict=defaultdict(int)
        for i in array:
            mydict[i[0]]+=1
        return np.sum( np.array( list(mydict.values())   ))
    
    #A teacher cant have clashes in classes
    def constraint1(self, convert):
        violation = 0
        days=np.unique(convert[:,5]).flatten()
        for day in days:
            index_days=np.argwhere(convert[:,5]==day)
            teachers=np.unique(convert[index_days,2]).flatten()
            sum1=0
            for teacher in teachers:
                index_teacher=np.argwhere(np.logical_and(convert[:,2]==teacher,convert[:,5]==day))
                sum1+=len(index_teacher)
                violation+=self.overlap(convert[index_teacher,[6,7]])
        return violation*-3
            
    
    #A batch_section cannot take two hour class of one course (in case of one hour class) or 3 hour class (in case of 1.5hr class) in a day.
    def constraint2(self, convert):
        violation=0
        days=np.unique(convert[:5]).flatten()
        for day in days:
            index_days=np.argwhere(convert[:,5]==day)
            groups=np.unique(convert[index_days,0]).flatten()
        
            for group in groups:
                index_group=np.argwhere(np.logical_and(convert[:,0]==group,convert[:,5]==day))
                violation+= self.same_course(convert[index_group,1])
        return violation*-1

    def constraint3(self, convert):
        violation=0
        days=np.unique(convert[:,5]).flatten()
        #print "days ",days
        #print days.shape
        for day in days:
            index_days=np.argwhere(convert[:,5]==day)
            #print "number of teachers ", len(index_days)
            rooms=np.unique(convert[index_days,4]).flatten()
            #print " teach ",teachers
            #sum1=0
            for room in rooms:
                index_room=np.argwhere(np.logical_and(convert[:,4]==room,convert[:,5]==day))
                #sum1+=len(index_room)
                violation+=self.overlap(convert[index_room,[6,7]])
            
            #print "sum1 ",sum1
        
            #print " vio ",violation
        return violation*-4
    
    def cross_over(self, chromosome_list):
        length=len(chromosome_list)
        i=0
        while i<length:
            temp1=chromosome_list[i]
            temp2=chromosome_list[i+1]
            split=np.random.randint(1,len(temp1)-1)
            temp3=np.vstack((temp1[:split],temp2[split:]))
            temp4=np.vstack((temp2[:split],temp1[split:]))
            chromosome_list[i]=temp3
            chromosome_list[i+1]=temp4
            i+=2
        return chromosome_list 
    
    def mutation(self, chromosome_list):
        for i in range(len(chromosome_list)):
            
            length=len(chromosome_list[0])
            idx=np.random.randint(length)
            chance=np.random.randint(0,100) #choose random
            temp1=chromosome_list[i]

            if chance<1:
                print ("Performing Mutation...")
                temp=temp1[idx]
                t=np.random.randint(0,2)
                if t==0:
                    a=np.random.choice(self.TotalRooms)
                    temp[4]=a
                    temp1[idx]=temp
                elif t==1:    
                    c=np.random.choice(self.time[:-1])
                    temp[6]=c
                    temp[7]=c+float(temp[3])
                    temp1[idx]=temp
            chromosome_list[i]=temp1
            
        return chromosome_list
    
    #From a chrosome, find if its to be selected or not
    def selection(self, chro):
        utility= self.EvaluateFitness(chro)
        sum1=np.sum(utility)
        number=np.random.randint(sum1)
        cummulative_sum=0
        for i in range(50):
            cummulative_sum+=utility[i]
            if number<cummulative_sum:
                return chro[i]  
    
    #function that loop over the entire population and selects the best fitness chromosomes
    def call_selection(self, chromosome_list):
        selected_lists=[]
        for i in range(len(chromosome_list)):
            selected_lists.append(self.selection(chromosome_list).reshape((381,8)))
        return selected_lists
    
    #Main Evaluation function
    def EvaluateFitness (self, Population):
        Constraint1 = self.getFitnessByConstraint(Population, 1) + 1143
        Constraint2 = self.getFitnessByConstraint(Population, 2) + 381
        Constraint3 = self.getFitnessByConstraint(Population, 3) + 1524
        return (Constraint1 + Constraint2 + Constraint3)
    
    #Gets the fitness of according to each contraint
    def getFitnessByConstraint (self, Population, option):
        FitnessArray = []
        for i in range(len(Population)):
            if (option == 1):
                FitnessArray.append(self.constraint3(Population[i]))
            elif (option == 2):
                FitnessArray.append(self.constraint2(Population[i]))
            elif (option == 3):
                FitnessArray.append(self.constraint3(Population[i]))
        #print "hello ",min(utility1)
        #print max(utility1)
        return np.array(FitnessArray)
    
    #Creates the main population
    def Populate(self):
        index = 0
        self.Population = []
        while (index<10):
            tempChromosomeArray = self.generateChromosome()
            self.Population.append(tempChromosomeArray)
            index = index + 1
        self.Population = np.array(self.Population)
        print (self.Population)
        return self.Population


# Reading Test Case

In [37]:
var = TimeTable_GA()
var.readTestFile("AI_testCase.xlsx")

Storing the Courses Offered:


,0
0,C1
1,C2
2,C3
3,C4
4,C5
5,C6


Storing the Classes that are to be scheduled: 


,0,1,2,3
0,BatchCS16_A,C1,T1,1.0
1,BatchCS16_A,C1,T1,1.0
2,BatchCS16_A,C1,T1,1.0
3,BatchCS16_A,C2,T2,1.5
4,BatchCS16_A,C2,T2,1.5
5,BatchCS16_A,C3,T3,1.0
6,BatchCS16_A,C3,T3,1.0
7,BatchCS16_A,C3,T3,1.0
8,BatchCS16_A,C4,T4,1.5
9,BatchCS16_A,C4,T4,1.5


Defining rooms


,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


Days in the week
[1 2 3 4 5]
Time Intervals
[  9.    9.5  10.   10.5  11.   11.5  12.   12.5  13.   13.5  14.   14.5
  15.   15.5  16. ]


#  Generating Population

In [38]:
chromosome_list = var.Populate()
print (len(chromosome_list))

0   ['BatchCS16_A ', 'C1', 'T1', '1.0', 5, 2, 9.0, 10.5]
1   ['BatchCS16_A ', 'C1', 'T1', '1.0', 8, 5, 15.0, 16.5]
2   ['BatchCS16_A ', 'C1', 'T1', '1.0', 14, 3, 13.5, 15.0]
3   ['BatchCS16_A ', 'C2', 'T2', '1.5', 14, 2, 9.5, 11.0]
4   ['BatchCS16_A ', 'C2', 'T2', '1.5', 1, 2, 13.5, 15.0]
5   ['BatchCS16_A ', 'C3', 'T3', '1.0', 13, 1, 14.5, 16.0]
6   ['BatchCS16_A ', 'C3', 'T3', '1.0', 15, 2, 12.5, 14.0]
7   ['BatchCS16_A ', 'C3', 'T3', '1.0', 7, 2, 11.0, 12.5]
8   ['BatchCS16_A ', 'C4', 'T4', '1.5', 9, 4, 9.0, 10.5]
9   ['BatchCS16_A ', 'C4', 'T4', '1.5', 9, 4, 11.5, 13.0]
10   ['BatchCS16_A ', 'C5', 'T5', '1.0', 7, 4, 13.5, 15.0]
11   ['BatchCS16_A ', 'C5', 'T5', '1.0', 1, 3, 9.5, 11.0]
12   ['BatchCS16_A ', 'C5', 'T5', '1.0', 1, 3, 14.0, 15.5]
13   ['BatchCS16_A ', 'C6', 'T6', '1.0', 9, 1, 11.0, 12.5]
14   ['BatchCS16_A ', 'C6', 'T6', '1.0', 4, 4, 14.5, 16.0]
15   ['BatchCS16_A ', 'C6', 'T6', '1.0', 1, 1, 13.0, 14.5]
16   ['BatchCS16_B', 'C1', 'T1', '1.0', 13, 3, 9.0, 10.5]
17   ['B

317   ['BatchCS13_D', 'C3', 'T39', '1.0', 15, 2, 9.5, 11.0]
318   ['BatchCS13_D', 'C4', 'T40', '1.0', 5, 2, 15.5, 17.0]
319   ['BatchCS13_D', 'C4', 'T40', '1.0', 4, 2, 12.0, 13.5]
320   ['BatchCS13_D', 'C4', 'T40', '1.0', 9, 4, 11.5, 13.0]
321   ['BatchCS13_D', 'C5', 'T41', '1.0', 2, 1, 9.0, 10.5]
322   ['BatchCS13_D', 'C5', 'T41', '1.0', 15, 4, 13.0, 14.5]
323   ['BatchCS13_D', 'C5', 'T41', '1.0', 9, 3, 15.5, 17.0]
324   ['BatchCS13_D', 'C6', 'T42', '1.0', 9, 2, 9.0, 10.5]
325   ['BatchCS13_D', 'C6', 'T42', '1.0', 11, 5, 15.5, 17.0]
326   ['BatchCS13_D', 'C6', 'T42', '1.0', 5, 2, 13.0, 14.5]
327   ['BatchCS13_E', 'C1', 'T37', '1.0', 3, 3, 13.5, 15.0]
328   ['BatchCS13_E', 'C1', 'T37', '1.0', 9, 1, 13.0, 14.5]
329   ['BatchCS13_E', 'C1', 'T37', '1.0', 15, 1, 14.0, 15.5]
330   ['BatchCS13_E', 'C2', 'T38', '1.0', 10, 1, 16.0, 17.5]
331   ['BatchCS13_E', 'C2', 'T38', '1.0', 1, 2, 15.0, 16.5]
332   ['BatchCS13_E', 'C2', 'T38', '1.0', 12, 3, 9.0, 10.5]
333   ['BatchCS13_E', 'C3', 'T39', '1.

291   ['BatchCS13_C', 'C1', 'T37', '1.0', 5, 1, 11.5, 13.0]
292   ['BatchCS13_C', 'C1', 'T37', '1.0', 8, 2, 13.0, 14.5]
293   ['BatchCS13_C', 'C1', 'T37', '1.0', 8, 3, 15.0, 16.5]
294   ['BatchCS13_C', 'C2', 'T38', '1.0', 2, 3, 15.5, 17.0]
295   ['BatchCS13_C', 'C2', 'T38', '1.0', 15, 5, 16.0, 17.5]
296   ['BatchCS13_C', 'C2', 'T38', '1.0', 2, 4, 13.0, 14.5]
297   ['BatchCS13_C', 'C3', 'T39', '1.0', 11, 5, 16.0, 17.5]
298   ['BatchCS13_C', 'C3', 'T39', '1.0', 3, 3, 9.5, 11.0]
299   ['BatchCS13_C', 'C3', 'T39', '1.0', 1, 1, 15.5, 17.0]
300   ['BatchCS13_C', 'C4', 'T40', '1.0', 14, 5, 10.5, 12.0]
301   ['BatchCS13_C', 'C4', 'T40', '1.0', 5, 5, 10.5, 12.0]
302   ['BatchCS13_C', 'C4', 'T40', '1.0', 15, 3, 12.5, 14.0]
303   ['BatchCS13_C', 'C5', 'T41', '1.0', 15, 2, 10.5, 12.0]
304   ['BatchCS13_C', 'C5', 'T41', '1.0', 6, 4, 16.0, 17.5]
305   ['BatchCS13_C', 'C5', 'T41', '1.0', 13, 2, 13.0, 14.5]
306   ['BatchCS13_C', 'C6', 'T42', '1.0', 1, 5, 11.0, 12.5]
307   ['BatchCS13_C', 'C6', 'T42', 

21   ['BatchCS16_B', 'C3', 'T3', '1.0', 3, 3, 12.5, 14.0]
22   ['BatchCS16_B', 'C3', 'T3', '1.0', 14, 4, 14.5, 16.0]
23   ['BatchCS16_B', 'C3', 'T3', '1.0', 4, 5, 11.5, 13.0]
24   ['BatchCS16_B', 'C4', 'T4', '1.5', 3, 5, 10.0, 11.5]
25   ['BatchCS16_B', 'C4', 'T4', '1.5', 12, 5, 14.5, 16.0]
26   ['BatchCS16_B', 'C5', 'T5', '1.0', 13, 1, 13.0, 14.5]
27   ['BatchCS16_B', 'C5', 'T5', '1.0', 15, 3, 16.0, 17.5]
28   ['BatchCS16_B', 'C5', 'T5', '1.0', 3, 3, 10.0, 11.5]
29   ['BatchCS16_B', 'C6', 'T6', '1.0', 7, 4, 11.5, 13.0]
30   ['BatchCS16_B', 'C6', 'T6', '1.0', 5, 2, 15.5, 17.0]
31   ['BatchCS16_B', 'C6', 'T6', '1.0', 7, 1, 12.5, 14.0]
32   ['BatchCS16_C', 'C1', 'T1', '1.0', 2, 4, 13.0, 14.5]
33   ['BatchCS16_C', 'C1', 'T1', '1.0', 11, 4, 13.0, 14.5]
34   ['BatchCS16_C', 'C1', 'T1', '1.0', 6, 1, 10.5, 12.0]
35   ['BatchCS16_C', 'C2', 'T2', '1.5', 3, 1, 12.0, 13.5]
36   ['BatchCS16_C', 'C2', 'T2', '1.5', 14, 2, 15.5, 17.0]
37   ['BatchCS16_C', 'C3', 'T3', '1.0', 4, 3, 10.5, 12.0]
38   ['B

145   ['BatchCS15_E', 'C2', 'T20', '1.0', 8, 4, 10.0, 11.5]
146   ['BatchCS15_E', 'C2', 'T20', '1.0', 3, 3, 11.0, 12.5]
147   ['BatchCS15_E', 'C3', 'T21', '1.0', 10, 5, 14.5, 16.0]
148   ['BatchCS15_E', 'C3', 'T21', '1.0', 10, 3, 14.5, 16.0]
149   ['BatchCS15_E', 'C3', 'T21', '1.0', 5, 2, 15.0, 16.5]
150   ['BatchCS15_E', 'C4', 'T22', '1.0', 6, 5, 15.0, 16.5]
151   ['BatchCS15_E', 'C4', 'T22', '1.0', 13, 2, 12.5, 14.0]
152   ['BatchCS15_E', 'C4', 'T22', '1.0', 10, 5, 9.5, 11.0]
153   ['BatchCS15_E', 'C5', 'T23', '1.0', 5, 5, 11.0, 12.5]
154   ['BatchCS15_E', 'C5', 'T23', '1.0', 5, 5, 16.0, 17.5]
155   ['BatchCS15_E', 'C5', 'T23', '1.0', 6, 4, 11.0, 12.5]
156   ['BatchCS15_E', 'C6', 'T24', '1.0', 6, 5, 9.0, 10.5]
157   ['BatchCS15_E', 'C6', 'T24', '1.0', 15, 1, 13.0, 14.5]
158   ['BatchCS15_E', 'C6', 'T24', '1.0', 15, 1, 10.5, 12.0]
159   ['BatchCS15_F', 'C1', 'T19', '1.0', 7, 5, 10.0, 11.5]
160   ['BatchCS15_F', 'C1', 'T19', '1.0', 4, 5, 11.0, 12.5]
161   ['BatchCS15_F', 'C1', 'T19', '

371   ['BatchCS16_F', 'C3', 'T45', '1.0', 4, 2, 12.0, 13.5]
372   ['BatchCS16_F', 'C4', 'T31', '1.0', 4, 1, 15.5, 17.0]
373   ['BatchCS16_F', 'C4', 'T31', '1.0', 14, 2, 9.0, 10.5]
374   ['BatchCS16_F', 'C4', 'T31', '1.0', 3, 3, 14.5, 16.0]
375   ['BatchCS16_F', 'C5', 'T32', '1.0', 11, 5, 12.0, 13.5]
376   ['BatchCS16_F', 'C5', 'T32', '1.0', 1, 1, 11.5, 13.0]
377   ['BatchCS16_F', 'C5', 'T32', '1.0', 2, 5, 11.5, 13.0]
378   ['BatchCS16_F', 'C6', 'T33', '1.0', 7, 4, 10.5, 12.0]
379   ['BatchCS16_F', 'C6', 'T33', '1.0', 4, 3, 9.0, 10.5]
380   ['BatchCS16_F', 'C6', 'T33', '1.0', 14, 4, 14.0, 15.5]
0   ['BatchCS16_A ', 'C1', 'T1', '1.0', 14, 1, 11.5, 13.0]
1   ['BatchCS16_A ', 'C1', 'T1', '1.0', 13, 2, 12.5, 14.0]
2   ['BatchCS16_A ', 'C1', 'T1', '1.0', 11, 4, 10.5, 12.0]
3   ['BatchCS16_A ', 'C2', 'T2', '1.5', 13, 5, 15.0, 16.5]
4   ['BatchCS16_A ', 'C2', 'T2', '1.5', 10, 2, 11.5, 13.0]
5   ['BatchCS16_A ', 'C3', 'T3', '1.0', 11, 2, 13.0, 14.5]
6   ['BatchCS16_A ', 'C3', 'T3', '1.0', 1, 4,

315   ['BatchCS13_D', 'C3', 'T39', '1.0', 11, 5, 12.0, 13.5]
316   ['BatchCS13_D', 'C3', 'T39', '1.0', 13, 5, 14.5, 16.0]
317   ['BatchCS13_D', 'C3', 'T39', '1.0', 6, 2, 12.5, 14.0]
318   ['BatchCS13_D', 'C4', 'T40', '1.0', 13, 3, 13.0, 14.5]
319   ['BatchCS13_D', 'C4', 'T40', '1.0', 8, 2, 15.5, 17.0]
320   ['BatchCS13_D', 'C4', 'T40', '1.0', 9, 2, 9.0, 10.5]
321   ['BatchCS13_D', 'C5', 'T41', '1.0', 8, 1, 11.5, 13.0]
322   ['BatchCS13_D', 'C5', 'T41', '1.0', 4, 5, 11.0, 12.5]
323   ['BatchCS13_D', 'C5', 'T41', '1.0', 8, 5, 11.0, 12.5]
324   ['BatchCS13_D', 'C6', 'T42', '1.0', 15, 4, 12.5, 14.0]
325   ['BatchCS13_D', 'C6', 'T42', '1.0', 15, 1, 13.5, 15.0]
326   ['BatchCS13_D', 'C6', 'T42', '1.0', 13, 2, 11.5, 13.0]
327   ['BatchCS13_E', 'C1', 'T37', '1.0', 2, 5, 9.5, 11.0]
328   ['BatchCS13_E', 'C1', 'T37', '1.0', 13, 2, 12.5, 14.0]
329   ['BatchCS13_E', 'C1', 'T37', '1.0', 15, 4, 9.5, 11.0]
330   ['BatchCS13_E', 'C2', 'T38', '1.0', 11, 2, 14.5, 16.0]
331   ['BatchCS13_E', 'C2', 'T38',

# Genetic Algorithm

In [39]:
max_value=0
idx=-1
for i in range(10):
    util= var.EvaluateFitness(chromosome_list)
    print ("max value ",np.max(util))
    if idx<0 or max_value<np.max(util):
        idx=np.argmax(util)
        max_value=np.max(util)
        print (max_value)
        print (chromosome_list[idx])
    print ("Performing Selection...")
    chromosome_list=var.call_selection(chromosome_list)
    print ("Performing CrossOver... ")
    chromosome_list=var.cross_over(chromosome_list)
   
    chromosome_list=var.mutation(chromosome_list)

max value  1033
1033
[['BatchCS16_A ' 'C1' 'T1' ..., '3' '9.5' '11.0']
 ['BatchCS16_A ' 'C1' 'T1' ..., '1' '9.5' '11.0']
 ['BatchCS16_A ' 'C1' 'T1' ..., '3' '13.5' '15.0']
 ..., 
 ['BatchCS16_F' 'C6' 'T33' ..., '3' '11.0' '12.5']
 ['BatchCS16_F' 'C6' 'T33' ..., '1' '10.0' '11.5']
 ['BatchCS16_F' 'C6' 'T33' ..., '2' '9.5' '11.0']]
Performing Selection...
Performing CrossOver... 
max value  1033
Performing Selection...
Performing CrossOver... 
Performing Mutation...
max value  949
Performing Selection...
Performing CrossOver... 
max value  1043
1043
[['BatchCS16_A ' 'C1' 'T1' ..., '3' '13.0' '14.5']
 ['BatchCS16_A ' 'C1' 'T1' ..., '2' '11.0' '12.5']
 ['BatchCS16_A ' 'C1' 'T1' ..., '1' '16.0' '17.5']
 ..., 
 ['BatchCS16_F' 'C6' 'T33' ..., '4' '10.5' '12.0']
 ['BatchCS16_F' 'C6' 'T33' ..., '3' '9.5' '11.0']
 ['BatchCS16_F' 'C6' 'T33' ..., '3' '15.5' '17.0']]
Performing Selection...
Performing CrossOver... 
max value  1119
1119
[['BatchCS16_A ' 'C1' 'T1' ..., '3' '13.0' '14.5']
 ['BatchCS16

In [35]:
print (max_value)

1129
